# TP4 - Análise Conformacional da α-Endorfina

Este trabalho tem por objectivo introduzir técnicas de simulação de Dinâmica Molecular usando Mecânica Molecular. Para este efeito, o trabalho encontra-se dividido em duas partes:
1. Preparação, condicionamento e corrida de uma simulação de dinâmica molecular de α-endorfina em meio aquoso.
2. Uso do VMD para análise da trajéctória e exploração das propriedades dinâmicas da α-endorfina.

Para a primeira parte será usado o pacote de software Tinker (disponível em https://dasher.wustl.edu/tinker/). Embora este pacote esteja disponível para Linux, MaxOS e Windows, a primeira parte deste trabalho será executada no ambiente Linux providenciado pela plataforma Google Colab.

A segunda parte do trabalho será levada a cabo usando o programa VMD (disponível em https://www.ks.uiuc.edu/Research/vmd/, após registo), o qual encontra-se instalado nos computadores da ECUM. 

In [ ]:
# instalação do Tinker
! wget https://dasher.wustl.edu/tinker/downloads/bin-linux-8.10.2.tar.gz
! tar xf bin-linux-8.10.2.tar.gz
! export PATH=/content/bin-linux:${PATH}
# O campo de forças amber99 será usado neste trabalho
! wget https://raw.githubusercontent.com/TinkerTools/tinker/release/params/amber99.prm

## Parte 1 - Simulação da α-endorfina em água.

A α-endorfina é um péptido com propriedades opióides libertado naturalmente pelo corpo em reacção à dor. A estrutura primária deste péptido é:

```TYR-GLY-GLY-PHE-MET-THR-SER-GLU-LYS-SER-GLN-THR-PRO-LEU-VAL-THR```


O primeiro passo é pois criar uma geometria inicial do péptido a partir da estrutura primária. Para isto, usa-se o comando `protein` do Tinker. Este comando pode correr de forma iterativa (fazendo perguntas ao utlizador na linha de comando) ou lendo um ficheiro com as instruções. Na célula seguinte, criamos as instruções para o `protein` (usando um pouco de código python) e depois executamos o comando (as linhas começadas pelo ponto de exclamação marcam que o comando é para ser interpretado pelo linux e não pelo python).

In [ ]:
with open('protein.conf','w') as f:
  f.write("""endorphin
alpha-endorphin
amber99.prm
TYR
GLY
GLY
PHE
MET
THR
SER
GLU
LYS
SER
GLN
THR
PRO
LEU
VAL
THR

N

""")

! /content/bin-linux/protein < protein.conf

Deverá agora ter quatro novos ficheiros: protein.conf (criado por nós), e três ficheiros chamados endorphin, com terminações diferentes: O ficheiro seq contém a sequência de aminoácidos, o ficheiro xyz contém as coordenadas cartesianas e topologia do péptido, e o ficheiro int contém a mesma informação mas em coordenadas internas (cada átomo A é definido por uma distância a outro átomo B, um ângulo a um terceiro átomo C e um diedro ABCD a um quarto átomo D). Pode descarregar o ficheiro `endorphin.xyz` e inspecionar a sua geometria no vmd. Para isso, **ao carregar a molécula deve certificar-se que o VMD a lê como Tinker e não como XYZ!**

O péptido agora criado é uma cadeia aproximadamente linear. Uma vez que (para efeitos deste excercício) partimos sem nenhuma ideia inicial da conformação da cadeia péptídica, devemos começar por fazer uma pesquisa do espaço conformacional (i.e. das rotações possíveis em torno das ligações simples). O Tinker oferece várias ferramentas para ajudar neste processo, sendo a `optirot` uma das mais rápidas. Mais uma vez, podemos correr a `optirot` de forma interactiva, ou fornecer todas as respostas num só comando.

In [ ]:
# Correr a optirot...
# ... para optimizar a partir da geometria contida em endorphin.int (o ficheiro xyz não funciona aqui)
# ... usando os parâmetros do campo de força do ficheiro amber99.prm
# ... determinar as rotações a explorar automaticamente (0)
# ... optimizar a geometria até um gradiente médio máximo de 0.01

!/content/bin-linux/optirot endorphin.int amber99.prm 0 0.01

No final, a `optirot` escreve novamente as coordenadas internas da α-endorfina. Como o ficheiro `endorphin.int` já existe no sistema, a `optirot` coloca um '_2' no final do seu ficheiro. Este é um comportamento transversal a todos os programas do pacote Tinker para evitar escrever por ciam de ficheiro existentes. Ainda assim, é conveniente no final de cada cálculo mudar o nome dos ficheiros de forma a melhor descrever o estado do trabalho aquando da criação do ficheiro.

Pode agora converter o ficheiro `endorphin.int_2` para o formato de coordenadas cartesianas do Tinker (primeiro mudando o nome para reflectir o estado do sistema), fazer download do ficheiro xyz entretanto formado e abrir no VMD.

In [ ]:
! mv endorphin.int_2 endorphin_optiroted.int
! /content/bin-linux/intxyz endorphin_optiroted.int

A geometria da α-endorfina presente no ficheiro `endorphin-optiroted.xyz` é o resultado de uma optimização no vácuo. No entanto, os péptidos podem alterar significativamente a sua conformação quando em contacto com a água. Para estudar o efeito da água, vamos colocar o nosso confórmero de α-endorfina numa caixa pré-equilibrada de moléculas de água, fornecida no ficheiro `water_box_30_30_30_amber99_equilibrated.xyz` (que descarregamos da net com o comando wget). Esta caixa contém 850 moléculas de água numa caixa cúbica com 30 Å de lado. Este ficheiro encontra-se já preparado de tar forma que as moléculas da água encontram-se organizadas de uma forma próxima da esperada para a água a 300 K.

Para incorporar a molécula da endorfina na caixa de água, usamos o programa `xyzedit` do Tinker. Neste comando vamos realizar duas operaçẽs: a primeira operação (13) desloca a molécula da α-endorfina de tar forma que o seu centro de massa fique na origem das coordenadas cartesianas. Depois, uma segunda operação (24) coloca a molécula da α-endorfina na caixa de moléculas de água, removendo as moléculas de água necessárias de forma a evitar colisões.

Finalmente, mudamos o nome do ficheiro `endorphin_optiroted.xyz_2` entretanto criado para `endorphin_solvated.xyz`. Poderá depois fazer download deste ficheiro e inspecionar a geometria da caixa no VMD.


In [ ]:
! wget https://raw.githubusercontent.com/teixeirafilipe/ChemOutReach/main/MM_MQM2223/water_box_30_30_30_amber99_equilibrated.xyz
! /content/bin-linux/xyzedit endorphin_optiroted.xyz amber99.prm 13 24 water_box_30_30_30_amber99_equilibrated.xyz
! mv endorphin_optiroted.xyz_2 endorphin_solvated.xyz

O programa `xyzedit` tenta fazer o seu melhor para evitar colisões entre as moléculas do soluto e do solvente. No entanto, a caixa que foi formada ainda pode ter tensões resíduais devido ao impacto que a endorfina tem na organização local da moléculas de água. Para resolver isto, devemos proceder a um cálculo de minimização de energia (optimização da geometria). O Tinker providencia várias utilidades para isso, sento a `minimize` uma das opções mais económicas a nível computacional. O comando `minimize` apenas requer um ficheiro com as coordenadas, um ficheiro com os parâmetros do campo de forças e um valor máximo aceitável para o gradiente da energia. Mais uma vez, após a minimização, é recomendado mudar o nome do ficheiro `endorphin_solvated.xyz_2` entrentanto criado.

In [ ]:
! time /content/bin-linux/minimize endorphin_solvated.xyz amber99.prm 0.01
! mv endorphin_solvated.xyz_2 endorphin_solv_min.xyz

O estudo da conformação da endorfina por Dinâmica Molecular (MD) é realizado em duas fases. Depois da optimização de geometria, as moléculas do sistema encontram-se num estado de muito baixa energia, pelo que é necessário introduzir energia cinética de forma a chegar à temperatura de produção. Só depois de atingido o equilíbrio térmico podemos finalmente começar o cálculo cujos resultados serão usados no nosso estudo.

Ambas as etapas são levadas a cabo pelo programa `dynamic` do Tinker, o qual tem como argumentos:
1. O ficheiro com as coordenadas
1. O ficheiro com os parâmetros do campo de força
1. O número de passos da simulação
1. O intervalo de tempo de cada passo (passo de integração), em fs
1. O intervalo de tempo para gravar a geometria do sistema no ficheiro de trajectória (ficheiro `.arc`)
1. A opção entre o ensemble NVE (1) ou NVT (2): usaremos sempre NVT
1. A temperatura desejada, em kelvin.

In [ ]:
! time /content/bin-linux/dynamic endorphin_solv_min.xyz amber99.prm 500 2.0 0.2 2 298

A estatística entre os passos 400 e 500 mostra que a temperatura encontra-se estabilizada nos 288 ± 4 K, pelo que podemos prosseguir para o cálculo de produção.

De forma a ter uma amostragem adequada do sistema, a as simulações de dinâmica molecular deverão tomar lugar ao longo de vários nanossegundos. Com um passo de integração de 1 fs, isto implica correr um milhão de passos por nanossegundo, o que (mesmo para um sistema pequeno como este) poderá demorar cerca de dois dias com os processadores da plataforma Colab. O cálculo de produção seguinte deverá demorar cerca de 25 minutos na plataforma Colab e cobrir 10 ps (ou 0.01 ns). **Deverá aproveitar o tempo da corrida para começar a responder a algumas das questões finais.**

Para correr o cálculo de produção tiramos proveito de o Tinker retomar o último ponto de uma simulação de MD caso encontre um ficheiro com o mesmo nome que o ficheiro das coordenadas, mas com a terminação `.dyn`. No entanto, não estamos interessados na trajéctória dos átomos na fase de equilibrio, pelo que removemos o ficheiro `endorphin_solv_min.arc` antes de re-iniciar a dinâmica com os parâmetros de produção (passo de integração mais curto, mais passos e gravação da trajéctória a intervalos de tempo mais curtos).

Para o cálculo de produção também vamos usar um "truque" do linux: o _output_ do programa `dynamic` vai ser levado por um `pipe` para o programa `tee` que escreve o output no ecrãn ao mesmo tempo que o grava num ficheiro (`endorphin_solv_md.out`) para podermos fazer estatísticas futuras com os dados produzidos pelo programa.

In [ ]:
! rm endorphin_solv_min.arc
! time /content/bin-linux/dynamic endorphin_solv_min.xyz amber99.prm 10000 1.0 0.1 2 298 | tee endorphin_solv_md.out

Antes de passar para a segunda parte do trabalho, temos que arquivar todo o conteúdo criado num ficheiro zip, que deverá transferir para o computador onde deseja realizar a análise. Antes disso, devemos converter a trajéctória da nossa simulação para o formato PDB, para que o VMD possa reconhecer os resíduos da cadeia péptidica. Vamos também aproveitar para extrair dados relativos à evolução da Temperatura, Energia cinética e energia potencial ao longo do tempo.

In [ ]:
! /content/bin-linux/xyzpdb endorphin_solv_min.arc amber99.prm
! mv endorphin_solv_min.pdb endorphin_solv_md.pdb

In [ ]:
# colectar dados do output da dinâmica molecular para um ficheiro csv
import pandas as pd
data=open("endorphin_solv_md.out",'r').readlines()
col=list()
pt={'Time_ps':None,'Frame':None,'Temperature_K':None,'Temperature_sd':None,'Etot_kJ_per_mol':None,'Epot_kJ_per_mol':None,'Ekin_kJ_per_mol':None}
for i,line in enumerate(data):
  if 'Temperature' in line: 
    pt['Temperature_K'] = float(line.split()[1])
    pt['Temperature_sd']= float(line.split()[-1].replace(')',''))
  if 'Frame Number' in line:
    pt['Frame']=int(line.split()[-1])
    pt['Time_ps']=float(data[i-3].split()[-2])
    pt['Epot_kJ_per_mol']=float(data[i-2].split()[-2])
    pt['Ekin_kJ_per_mol']=float(data[i-1].split()[-2])
    pt['Etot_kJ_per_mol'] = pt['Epot_kJ_per_mol']+pt['Ekin_kJ_per_mol']
    col.append(pt)
    pt={'Time_ps':None,'Frame':None,'Temperature_K':None,'Temperature_sd':None,'Etot_kJ_per_mol':None,'Epot_kJ_per_mol':None,'Ekin_kJ_per_mol':None}
col = pd.DataFrame(col)
col.to_csv('endorphin_solv_md.csv',index=False)

In [ ]:
# finalmente fazer um ficheiro zip com todos os ficheiros relevantes
! zip tp4-endorphin_md.zip endorphin* protein.conf amber99.prm water_box_30_30_30_amber99_equilibrated.xyz

Pode agora descarregar o ficheiro `tp4-endorphin_md.zip`, descompactar e fazer a análise da trajectória com o VMD.

## Parte 2: Análise da Trajéctória com o VMD

### Análise visual preliminar

1. Abra o ficheiro `endorphin_solv_md.pdb` com o VMD. Certifique-se que o VMD reconhece o ficheiro como `PDB`.
1. Use os comandos de animação na parte inferior da janela principal para visualizar a animação da trajectória.
1. Pause a simulação e use a caixa de diálogo `Representations...` para criar uma representação adequada para o estudo da cadeia péptídica (i.e. sem representar as moléculas de água).

### Estudo das pontes de H
1. Abra a caixa de diálogo `Hydrogen Bonds` que poderá encontrar nos menus da janela principal em `Extensions->Analysis`
1. Estudo das pontes de H dentro da cadeia peptídica.
  1. Verifique que tem a trajectória carregada no campo `Molecule` e que a `Selection 1` tem o conteúdo `protein`. Desta forma irá estudar as pontes de H dentro da cadeia peptídica. 
  1. Seleccione "Calculate Detailed Info for **All hbonds**"
  1. Verifique a opção "Write output to files". Dê nomes apropriados para o ficheiro com informação sobre nº de ligações em cada *frame* ("frame/bond data"), assim como do ficheiro com informação detalhada sobre as ligações.
  1. Clique em "Find hydrogen bonds!"
1. Estudo das pontes de H entre a cadeia peptídica e a água.
  1. Mantendo as definições anteriores, coloque "waters" no campo `Selection 2`.
  1. Modifique o nome dos ficheiros de *output* para evitar perder a informação das pontes de H internas.
  1. Clique em "Find hydrogen bonds!"
1. Em ambos os casos, poderá usar o ficheiro "frame/bond data" para recriar no Excel a evolução do número de pontes de H com o tempo da simulação. Poderá também usar o ficheiro com informação detalhada para encontrar pares dador-aceitador de pontes de H mais proeminentes.

### Análise da distribuição radial
O grupo OH do resíduo 16 é particularmente activo na formação e quebre da pontes de H. Podemos verificar quão estável é a esfera de solvatação deste resíduo através do gráfico da distribuição das moléculas de água em torno do oxigénio do grupo funcional em causa.
1. Na Janela Principal do VMD, abra `Extensions->Analysis->Radial Pair Distribution Function g(r)`
1. Selecione a molécula/ficheiro adequado na caixa `Use Molecule` 
1. Selecione o oxigénio do residuo 16 na caixa `Selection 1`, usando as condições: `(residue = 15) and (name is "OG1")`
1. Na caixa `Selection 2` seleccione todos os átomos de H da molécula de água: `name is "H"`
1. No menu `Utilities` verifique que os parâmetros da célula são os usados na simulação: a = b = c = 30 Å e α = β = γ = 90°.
1. Verifique que a opção "Display g(r)" está seleccionada.
1. Clique em "Compute g(r)"
1. Ajuste o parâmetro `delta r` a gosto.
1. No final, guarde os valores de \(r\) e \(g(r)\) a partir da janela do gráfico de \(g(r)\) em `File->Export to ASCII matrix...`. Poderá usar este ficheiro para fazer novamente o gráfico no excel.

### Visualização de propriedades por resíduo
1. Na janela principal do VMD, abra `Extensions->Analysis->Timeline`.
1. Escolha `Calculate->Calc. Sec. Struct.` para calcular a evolução da estrutura secundário, por aminoácido ao longo da trajectória. O VMD usa o sistema de classificação DSSP, no qual:
  * G = hélice \(3_{10}\) (mínimo de 3 resíduos)
  * H = Hélice-α (mínimo de 4 resíduos)
  * I = Hélice-π (mínimo de 5 resíduos)
  * T = *turn*
  * E = 2 ou mais resíduos em folha-β (paralela ou anti-paralela)
  * B = resíduo isolado em folha-β
  * S = *bend* (não envolve pontes de H)
  * C = *coil* (nenhuma das opções anteriores)
1. Pode guardar a imagem do mapa em `File->Print to File...`
1. Escolha `Calculate->Calc. Delta Phi` para visualizar onde ocorrem as maiores alterações do ângulo Φ.
1. Escolha `Calculate->Calc. Delta Psi` para visualizar onde ocorrem as maiores alterações do ângulo Ψ.

### Visualização do gráfico de Ramachandram
1. Na janela principal do VMD, abra `Extensions->Analysis->Ramachandram Plot`. As regiões de maior estabilidade em ordem a Φ e Ψ encontram-se marcadas a verde e azul.
1. Selecione o ficheiro molécula/trajectória na caixa `Molecule`. A posição média de cada aminoácido aparecerá no gráfico como caixas amarelas.
1. Clicando em qualquer uma dessas caixas permitirá visualizar as várias conformações do resído ao longo da trajectória.
1. Poderá gravar o gráfico como imagem em  `File->Print to File...`
1. Caso peça para fazer um histograma em 3D, este irá aparecer na janela de Display do VMD.

## Questões
1. Qual o estado de protonação dos terminais C e N do péptido gerado pelo comando `protein`?
1. Caso pretendesse incorporar iões Na$^+$ e Cl$^-$ para simular condições fisiológicas, quantos iões sódio deveria adicionar à caixa (assuma que a concentração fisiológica de NaCl é 1g/100mL)? Comente o valor obtido.
1. Calcule o pH do meio, caso tivesse um protão (H$^+$) na caixa da simulação (assuma que a proteína não tem actividade ácido-base). Comente este valor.
1. Coloque a representação que usou para a análise visual preliminar.
1. Represente graficamente a variação da temperatura e da energia total ao longo da simulação. Como interpreta estes gráficos?
1. Do estudo das pontes de H, qual o par de resíduos que passa mais tempo envolvido em pontes de H? Nesse par, qual é o aceitador e qual é o dador?
1. Qual o redíduo de aminoácido com maior propenção para formar pontes de H com a água envolvente?
1. Como interpreta os "picos" da distribuição radial dos átomos de H em relação ao oxigénio do grupo OH do resíduo 16.
1. Quais os resíduos de aminoácido que estão fora das regiões de estabilidade conformacional. Destes, quais poderão tenter no futuro para uma dessas regiões, e quais aparentam continuar fora delas. Justifique porque só dois aminoácidos caem nesta última categoria.
